In [20]:
#read the 2 csvs.
import numpy as np
import pandas as pd
import re
df_train = pd.read_csv("data/Corona_NLP_train.csv",encoding='latin2')
df_test  = pd.read_csv("data/Corona_NLP_test.csv",encoding='latin2')

In [21]:
#turn Extremely Positive/Negative into Positive/Negative.
df_train.loc[df_train['Sentiment']=='Extremely Positive', 'Sentiment'] = 'Positive'
df_train.loc[df_train['Sentiment']=='Extremely Negative', 'Sentiment'] = 'Negative'
df_test.loc[df_test['Sentiment']=='Extremely Positive', 'Sentiment'] = 'Positive'
df_test.loc[df_test['Sentiment']=='Extremely Negative', 'Sentiment'] = 'Negative'
#delete the Neutral values
df_train=df_train[-df_train.Sentiment.isin(['Neutral'])]
df_test=df_test[-df_test.Sentiment.isin(['Neutral'])]

In [22]:
#incert a new column "Label" which reflects the "Sentiment" column. "Positive" is 1 and "Negetive" is 0.
df_train.insert(df_train.shape[1], 'Label', 0)
df_train.loc[df_train['Sentiment']=='Positive', 'Label'] = 1
df_test.insert(df_test.shape[1], 'Label', 0)
df_test.loc[df_test['Sentiment']=='Positive', 'Label'] = 1

In [23]:
df_train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment,Label
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive,1
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive,1
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive,1
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Negative,0
5,3804,48756,"ĂT: 36.319708,-82.363649",16-03-2020,As news of the regionÂs first confirmed COVID...,Positive,1


In [24]:
def preprocess_text(text):
    text = ' '.join(word.lower() for word in text.split(" "))
    text = re.sub(r"([.,!?])", r" \1 ", text)
    text = re.sub(r"[^a-zA-Z.,!?]+", r" ", text)
    return text

df_train.OriginalTweet =df_train.OriginalTweet.apply(preprocess_text)
df_test.OriginalTweet =df_test.OriginalTweet.apply(preprocess_text)

In [25]:
from sklearn.model_selection import train_test_split
df_train = df_train[['OriginalTweet','Sentiment']]
test = df_test[['OriginalTweet','Sentiment']]
train, val = train_test_split(df_train, test_size=0.2)

In [26]:
train['split'] = 'train'
test['split'] = 'test'
val['split'] = 'val'
result = pd.concat([train,test,val]).reset_index(drop = True)

C:\Users\lucia\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\lucia\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\lucia\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in

In [27]:
result.head()

,OriginalTweet,Sentiment,split
0,britain s frontline minimum wage army got a pa...,Positive,train
1,a perfect storm us facing hunger crisis as de...,Negative,train
2,coles woolworths lgnsw ritapanahi alanjones s...,Negative,train
3,oil prices rise as trump expects russia and sa...,Negative,train
4,where are your prices gouging police for exced...,Negative,train


In [29]:
result.to_csv('data/Corona_NLP_final.csv', index=False)